In [ ]:
###########################################################################
# I would like to acknowledge the authors of these task and images.
# Kaggle problem: https://www.kaggle.com/andrewmvd/face-mask-detection
# Source of the data: https://makeml.app/datasets/mask
###########################################################################

In [1]:
#### 
# Modify the model_num according to the model to be used.
####
# 1 = PCA SVM
# 2 = RandomForest
# 3 = CNN
model_num = 3

In [2]:
!pip install tensorflow==2.0

In [3]:
!pip install keras==2.3.0

In [4]:
!pip install opencv-python

In [5]:
import os, sys
#import pandas as pd
import numpy as np
import cv2

#import matplotlib as mpl
#import matplotlib.pyplot as plt

In [6]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [7]:
#### 
# Modify the model_num according to the model to be used.
####
# 1 = PCA SVM
# 2 = RandomForest
# 3 = CNN

import pickle as pk

if model_num == 1:
    # Load PCA SVM model
    pca_model = pk.load(open(r"C:\Users\vigne\OneDrive\Desktop\adv_ds_project\pca_model.pkl",'rb'))
    svm_model = pk.load(open(r"C:\Users\vigne\OneDrive\Desktop\adv_ds_project\svm_pca_model.pkl",'rb'))
    # result_new = pca_reload .transform(X)
elif model_num == 2:
    # Load RandomForest model
    rf_model = pk.load(open(r"C:\Users\vigne\OneDrive\Desktop\adv_ds_project\rf_model.pkl",'rb'))
else:
    # Load CNN model
    cnn_model = load_model(r"C:\Users\vigne\OneDrive\Desktop\adv_ds_project\CNN_model_014.model")
    #cnn_model = pk.load(open(r"C:\Users\vigne\OneDrive\Desktop\adv_ds_project\cnn_model.pkl",'rb'))

In [8]:
# 0 = mask worn wrongly 1 = with mask  2 = without mask
labels = {0:'Mask worn wrongly',1:'With mask',2:'Without mask'}
colours = {0:(255,0,0),1:(0,255,0),2:(0,0,255)}
size=100

In [9]:
# Import face detector algorithm
face_detector = cv2.CascadeClassifier(r"C:\Users\vigne\OneDrive\Desktop\adv_ds_project\haarcascade_frontalface_default.xml")

In [10]:
src = cv2.VideoCapture(0)
while(True):
    ret,img = src.read()
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img_gray,1.3,5)
    
    for x,y,width,height in faces:
        
        # Extract face
        face = img_gray[y:y+height+20,x:x+width]
        # Resize to 100 by 100
        face_resized = cv2.resize(face,(size,size))
        
        # Reshape according to ML/DL
        if model_num == 3:
            face_resized = np.reshape(face_resized,(1, size, size, 1))           
        else:   
            face_resized = np.reshape(face_resized,(1,size*size))
        
        # Normalize
        face_resized_norm = face_resized/255.0
        
        
        if model_num == 1:
            # PCA SVM model
            face_resized_pca = pca_model.transform(face_resized_norm)
            face_pred = svm_model.predict(face_resized_pca)[0]
        elif model_num == 2:
            # RandomForest model
            face_pred  = rf_model.predict(face_resized_norm)[0]
        else:
            # CNN model
            face_pred_0  = cnn_model.predict(face_resized_norm)
            face_pred = np.argmax(face_pred_0, axis=1)[0]
          
        
        # Draw coloured rectangle to indicate face detection & label
        cv2.rectangle(img,(x,y),(x+width,y+height),colours[face_pred],2)
        cv2.rectangle(img,(x,y-40),(x+width,y),colours[face_pred],-1)
        cv2.putText(img,labels[face_pred],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('LIVE',img)
    exit = cv2.waitKey(1)
    if (exit==27): # 27 == ACSII char for 'ESC'
        break
        
cv2.destroyAllWindows()
src.release()